### Training a Multilayer CNN on MNIST data
Based on tutorial available at https://www.tensorflow.org/get_started/mnist/pros

In [19]:
# Load TF and the MNIST data
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
data_dir = "data_dir"
mnist = input_data.read_data_sets(data_dir, one_hot=True)

Extracting data_dir/train-images-idx3-ubyte.gz
Extracting data_dir/train-labels-idx1-ubyte.gz
Extracting data_dir/t10k-images-idx3-ubyte.gz
Extracting data_dir/t10k-labels-idx1-ubyte.gz


In [20]:
# Define functions for automatically creating weights and biases
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# Functions for automatically creating convolutional and pooling layers
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [21]:
# Creating the first layer: input -> convolution -> RELU -> pooling
x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1, 28, 28, 1]) # -1 tells TF to figure out that dimension automatically
W_conv1 = weight_variable([5, 5, 1, 32]) # receptive fields are 5-by-5
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [22]:
# Second layer: convolution -> RELU -> pooling
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [23]:
# Third layer: fully connected layer
W_fc1 = weight_variable([7 * 7 * 64, 1024]) # enough weights to connect all pairs of units
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [24]:
# Dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [25]:
# Fourth layer: fully connected layer -> output
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [26]:
# Build the cross-entropy, placeholder for true labels and accuracy.
ytrue = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=ytrue, logits=y_conv))
# Adam optimizer: https://arxiv.org/abs/1412.6980
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(ytrue, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [27]:
nstep=1000 #Set this to more like 20K for better accuracy on test
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(nstep):
        batch = mnist.train.next_batch(50)
        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict={
                x: batch[0], ytrue: batch[1], keep_prob: 1.0}) # keep_prob=1:No dropout during evaluation.
            print('step %d, training accuracy %g' % (i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], ytrue: batch[1], keep_prob: 0.5})
    print('test accuracy %g' % accuracy.eval(feed_dict={
            x: mnist.test.images, ytrue: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.06
step 100, training accuracy 0.84
step 200, training accuracy 0.92
step 300, training accuracy 0.96
step 400, training accuracy 0.98
step 500, training accuracy 0.94
step 600, training accuracy 1
step 700, training accuracy 0.92
step 800, training accuracy 0.94
step 900, training accuracy 0.88
test accuracy 0.9578


In [ ]:
# With nsteps set suitably high, accuracy should be about .99